In [70]:
# 새의 품종을 인식-- 
# 데이터 셋 : cub200-2011(200 새 영상 11788장)

In [75]:
# ImageNet으로 학습된 ResNet50을 cub 데이터셋으로 전이학습
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten,Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.resnet50 import ResNet50,preprocess_input
from tensorflow.keras.preprocessing import image
import os
import glob
import zipfile

In [88]:
train_foler = '/content/drive/MyDrive/Colab Notebooks/CUB200/train'
test_foler = '/content/drive/MyDrive/Colab Notebooks/CUB200/test'

In [89]:
zippath = '/content/drive/MyDrive/Colab Notebooks'

In [90]:
zippath

'/content/drive/MyDrive/Colab Notebooks'

In [91]:
# 압축해제
zipfile.ZipFile(zippath+"/CUB200.zip").extractall(zippath+"/CUB200")

In [92]:
# 데이터의 양을 줄여서 속도 증가
class_reduce = 0.1
no_class = int(len(os.listdir(train_foler))*class_reduce)

In [93]:
no_class

20

In [94]:
glob.glob(train_foler+"/*.*")[:20]

['/content/drive/MyDrive/Colab Notebooks/CUB200/train/001.Black_footed_Albatross',
 '/content/drive/MyDrive/Colab Notebooks/CUB200/train/002.Laysan_Albatross',
 '/content/drive/MyDrive/Colab Notebooks/CUB200/train/003.Sooty_Albatross',
 '/content/drive/MyDrive/Colab Notebooks/CUB200/train/004.Groove_billed_Ani',
 '/content/drive/MyDrive/Colab Notebooks/CUB200/train/005.Crested_Auklet',
 '/content/drive/MyDrive/Colab Notebooks/CUB200/train/006.Least_Auklet',
 '/content/drive/MyDrive/Colab Notebooks/CUB200/train/007.Parakeet_Auklet',
 '/content/drive/MyDrive/Colab Notebooks/CUB200/train/008.Rhinoceros_Auklet',
 '/content/drive/MyDrive/Colab Notebooks/CUB200/train/009.Brewer_Blackbird',
 '/content/drive/MyDrive/Colab Notebooks/CUB200/train/010.Red_winged_Blackbird',
 '/content/drive/MyDrive/Colab Notebooks/CUB200/train/011.Rusty_Blackbird',
 '/content/drive/MyDrive/Colab Notebooks/CUB200/train/012.Yellow_headed_Blackbird',
 '/content/drive/MyDrive/Colab Notebooks/CUB200/train/013.Bobolink

In [95]:
# 훈련데이터 읽기
x_train,y_train = [],[]
for i, class_name in  enumerate(glob.glob(train_foler+"/*.*")):
  if i< no_class:
    for fname in glob.glob(class_name+"/*.jpg"):
      img = image.load_img(fname,target_size=(244,244))
      x = image.img_to_array(img)
      x = preprocess_input(x)
      x_train.append(x)
      y_train.append(i)

In [96]:
# 테스트데이터 읽기
x_test,y_test = [],[]
for i, class_name in  enumerate(glob.glob(test_foler+"/*.*")):
  if i< no_class:
    for fname in glob.glob(class_name+"/*.jpg"):
      img = image.load_img(fname,target_size=(244,244))
      x = image.img_to_array(img)
      x = preprocess_input(x)
      x_test.append(x)
      y_test.append(i)

In [97]:
len(x_train), len(y_train), len(x_test), len(y_test)

(600, 600, 515, 515)

In [108]:
list1 = [2,4,6,8,9,10,12]
for data in list1:  
  try:
    assert data % 2 == 0
    print(data)
  except Exception as e:    
    print(e) 

2
4
6
8

10
12


In [98]:
x_train = np.asarray(x_train)
y_train = np.asarray(y_train)
x_test = np.asarray(x_test)
y_test = np.asarray(y_test)

In [99]:
x_train.shape

(600, 244, 244, 3)

In [100]:
base_model = ResNet50(weights='imagenet', include_top=False,input_shape=(244,244,3))
base_model.trainable = False  # 가중치를 동결

In [ ]:
base_model.summary()

In [102]:
cnn = Sequential()
cnn.add(base_model)
cnn.add(Flatten())
cnn.add(Dense(1024,activation = 'relu'))
cnn.add(Dense(no_class,activation='softmax'))

In [ ]:
cnn.summary()

In [104]:
# 컴파일
# learning_rate=0.00002  -> 미세조정방식의 학습(낮은학습률 설정)
# 2e-05  0.00002
cnn.compile(loss=tf.keras.losses.sparse_categorical_crossentropy,optimizer=Adam(learning_rate=0.00002), metrics=['accuracy'])

In [ ]:
# 학습
hist = cnn.fit(x_train,y_train,batch_size=16, epochs=10,validation_data=[x_test,y_test])

In [106]:
res = cnn.evaluate(x_test,y_test)
res

17/17 [==============================] - 2s 127ms/step - loss: 1.1920 - accuracy: 0.6971


[1.191971778869629, 0.6970874071121216]

In [107]:
# 미세조정방식  
#-컨볼류션층과 완정연결층의 가중치를 동시에 수정
#학습률은 낮게 유지하면서 조금씩 수정

# 동결방식
# 컨볼류션층과 완정연결층의 가중치를 동결해서 수정이 일어나지  않게 제한
# base_model.trainable = False

# 지금데이터셋에서는 미세조정방식이 조금 우수